In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, cross_validation
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [266]:
### Reading data

data = pd.read_csv('../data/data.csv')
data.set_index('shot_id', inplace=True)
# The following should be explicitly made categorical as they are encoded using integers
data["action_type"] = data["action_type"].astype('object')
data["game_event_id"] = data["game_event_id"].astype('category')
data["game_id"] = data["game_id"].astype('category')
data["period"] = data["period"].astype('object')
data["playoffs"] = data["playoffs"].astype('category')
data["season"] = data["season"].astype('category')
data["shot_made_flag"] = data["shot_made_flag"].astype('category')
data["shot_type"] = data["shot_type"].astype('category')
data["team_id"] = data["team_id"].astype('category')

In [445]:
# data cleaning
##
unknown_mask = data['shot_made_flag'].isnull()
data_cl = data.copy()
target = data_cl['shot_made_flag'].copy()
data_cl.drop('team_id', inplace=True, axis=1)  # only 1 category
data_cl.drop('lat', inplace=True, axis=1)  # correlated with loc_x
data_cl.drop('lon', inplace=True, axis=1)  # correlated with loc_y
data_cl.drop('game_id', inplace=True, axis=1)  # should not be dependent on game id
data_cl.drop('game_event_id', inplace=True, axis=1)  # independent, unique for every shots in a game
data_cl.drop('team_name', inplace=True, axis=1)  # always LA Lakers
data_cl.drop('shot_made_flag', inplace=True, axis=1)  # target variables
#data_cl.drop('combined_shot_type', inplace=True, axis=1)  # target variables

In [446]:
# Game date -- extract year and month
data_cl['game_date'] = pd.to_datetime(data_cl['game_date'])
data_cl['game_year'] = data_cl['game_date'].dt.year
data_cl['game_month'] = data_cl['game_date'].dt.month
data_cl.drop('game_date', axis=1, inplace=True)

In [447]:
#Extract time related features
data_cl['time_remaining'] = 60 * data_cl['minutes_remaining'] + data_cl['seconds_remaining']
data_cl['last_3_sec'] = data_cl['time_remaining'] < 3
data_cl['last_5_sec'] = data_cl['time_remaining'] < 5
data_cl['last_10_sec'] = data_cl['time_remaining'] < 10

# drop redundant features
data_cl.drop('minutes_remaining', axis=1, inplace=True)
data_cl.drop('seconds_remaining', axis=1, inplace=True)
data_cl.drop('time_remaining', axis=1, inplace=True)

In [448]:
from sklearn import mixture
gaussianMixtureModel1 = mixture.GMM(n_components= 16, covariance_type='full', params='wmc', init_params='wmc',
                                    random_state=1, n_init=3,  verbose=0)
gaussianMixtureModel1.fit(data_cl.ix[:,['loc_x','loc_y']])
data_cl['shotLocationCluster'] = gaussianMixtureModel1.predict(data_cl.ix[:,['loc_x','loc_y']])

data_cl.drop('loc_x', axis=1, inplace=True)
data_cl.drop('loc_y', axis=1, inplace=True)

In [449]:
## Matchup -- (away/home)
data_cl['home_play'] = data_cl['matchup'].str.contains('vs').astype('int')
data_cl.drop('matchup', axis=1, inplace=True)

In [450]:
data_cl['first_period'] = data_cl['period'] == 1
data_cl['second_period'] = data_cl['period'] == 2
data_cl['third_period'] = data_cl['period'] == 3
data_cl['fourth_period'] = data_cl['period'] == 4
data_cl['final_period'] = data_cl['period'] > 4
data_cl.drop('period', axis=1, inplace=True)

In [451]:
# Replace 20 least common action types with value 'Other'
rare_action_types = data_cl['action_type'].value_counts().sort_values().index.values[:20]
data_cl.loc[data_cl['action_type'].isin(rare_action_types), 'action_type'] = 'Other'

In [452]:
# Replace  least common action types with value 'rare'
rare_opponent = data_cl['opponent'].value_counts().sort_values().index.values[:30]
data_cl.loc[data_cl['opponent'].isin(rare_opponent), 'opponent'] = 'rare'

In [453]:
# Replace  least common action types with value 'rare'
rare_st = data_cl['combined_shot_type'].value_counts().sort_values().index.values[:2]
data_cl.loc[data_cl['combined_shot_type'].isin(rare_st), 'combined_shot_type'] = 'rare_st'

In [454]:
data_cl.columns

Index(['action_type', 'combined_shot_type', 'playoffs', 'season',
       'shot_distance', 'shot_type', 'shot_zone_area', 'shot_zone_basic',
       'shot_zone_range', 'opponent', 'game_year', 'game_month', 'last_3_sec',
       'last_5_sec', 'last_10_sec', 'shotLocationCluster', 'home_play',
       'first_period', 'second_period', 'third_period', 'fourth_period',
       'final_period'],
      dtype='object')

In [455]:
data_cl.describe()

,shot_distance,game_year,game_month,shotLocationCluster,home_play
count,30697.000000,30697.000000,30697.000000,30697.000000,30697.000000
mean,13.437437,2006.098446,5.578460,6.776688,0.487214
std,9.374189,4.835462,4.155881,4.442815,0.499845
min,0.000000,1996.000000,1.000000,0.000000,0.000000
25%,5.000000,2002.000000,2.000000,3.000000,0.000000
50%,15.000000,2006.000000,4.000000,6.000000,0.000000
75%,21.000000,2010.000000,11.000000,10.000000,1.000000
max,79.000000,2016.000000,12.000000,15.000000,1.000000


In [456]:
# One-hot encoding of categorical variables
categorial_cols = [
    'action_type', 'combined_shot_type', 'playoffs', 'season', 'shot_type',
    'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'opponent', 'shotLocationCluster']

for cc in categorial_cols:
    dummies = pd.get_dummies(data_cl[cc])
    dummies = dummies.add_prefix("{}#".format(cc))
    data_cl.drop(cc, axis=1, inplace=True)
    data_cl = data_cl.join(dummies)

In [457]:
# Train/validation split
# Separate dataset for validation
data_submit = data_cl[unknown_mask]
# Separate dataset for training
X = data_cl[~unknown_mask]
Y = target[~unknown_mask]

In [458]:
print('Clean dataset shape: {}'.format(data_cl.shape))

Clean dataset shape: (30697, 117)


In [459]:
regr = LogisticRegression(C=0.7, penalty='l1')
predicted = cross_validation.cross_val_predict(regr, X, Y, cv=10)
print(metrics.accuracy_score(Y, predicted))
print(metrics.classification_report(Y, predicted))

0.66474685761
             precision    recall  f1-score   support

        0.0       0.65      0.84      0.74     14232
        1.0       0.69      0.44      0.54     11465

avg / total       0.67      0.66      0.65     25697



In [460]:
data_submit = data_cl[unknown_mask]
regr.fit(X, Y)
preds = regr.predict_proba(data_submit)

In [461]:
submission = pd.DataFrame()
submission["shot_id"] = data_submit.index
submission["shot_made_flag"]= preds[:,1]
submission["shot_made_flag"]= submission["shot_made_flag"]

submission.to_csv("../data/sub.csv",index=False)